# Smartphone Price Prediction

In [9]:
!pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/24/ec/ad387100fa3cc2b9b81af0829b5ecfe75ec5bb19dd7c19d4fea06fb81802/xgboost-2.0.3-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB 187.9 kB/s eta 0:08:51
   ---------------------------------------- 0.0/99.8 MB 187.9 kB/s eta 0:08:51
   ---------------------------------------- 0.1/99.8 MB 273.8 kB/s eta 0:06:05
   ---------------------------------------- 0.1/99.8 MB 309.1 kB/s eta 0:05:23
   ---------------------------------------- 0.1/99.8 MB 361.0 kB/s eta 0:04:36
   ---------------------------------------- 0.2/99.8 MB 567.2 kB/s eta 0:02:56
   ---------------------------------------- 0.5/99.8 MB 1.1 MB/s eta 0:01:28
   -------------------------------

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#imported necessary libraries

In [6]:
#removes the limit from the number of displayed columns and rows
#this is so I can see the entire dataframe when I print it
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",100)

In [7]:
#to buid the linear model for statistical analysis and prediction
import statsmodels.stats.api as sms
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    AdaBoostRegressor,
    GradientBoostingRegressor,
    StackingRegressor,
)


### We  will use Linear Regression,Decision Tree Regressor,Random Forest Regressor,Ada Booost, Gradient Boosting Regressor,Stacking Regressor.
#### Stacking Regressor is a heterogeneous ensemble of different models(combination). Like Random Forest is an ensemble of multiple Decision Trees. If I wannt to combine a random forest with ada boost and gradient boosing then it will generate a Stacking model. If one of the algorithm is not giving the result then can I combine all of them to get a good result ?

In [10]:
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor #boosting techniques

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

### DATA LOADING AND EXPLORING

In [14]:
df=pd.read_csv("C:/Users/shrey/Downloads/smartphones.csv")
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")
#f string


There are 980 rows and 22 columns.


In [16]:
np.random.seed(1) #to get same random results every time
df.sample(n=10)

,brand_name,model,price,avg_rating,5G_or_not,processor_brand,num_cores,processor_speed,battery_capacity,fast_charging_available,fast_charging,ram_capacity,internal_memory,screen_size,refresh_rate,num_rear_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,resolution_height,resolution_width
370,oppo,OPPO Find N Flip,89990,8.8,1,dimensity,8.0,3.05,4300.0,1,44.0,8,128,6.80,120,2,NaN,50.0,32.0,0,2400,1200
692,sony,Sony Xperia 5 II,69990,8.6,1,snapdragon,8.0,2.84,4000.0,1,21.0,8,128,6.10,120,3,android,12.0,8.0,1,2520,1080
403,oppo,OPPO Reno 9 Z,22990,8.4,1,snapdragon,8.0,2.20,4800.0,1,33.0,8,128,6.58,120,3,android,50.0,32.0,1,2400,1080
678,samsung,Samsung Galaxy S23 Plus,84990,8.9,1,snapdragon,8.0,3.20,4700.0,1,45.0,8,256,6.60,120,3,android,50.0,12.0,0,2340,1080
966,xiaomi,Xiaomi Redmi Note 12 Pro Plus (6GB RAM + 128GB),26999,8.4,1,dimensity,8.0,2.60,4980.0,1,120.0,6,128,6.67,120,3,android,200.0,16.0,0,2400,1080
175,itel,itel S16,5990,NaN,0,unisoc,4.0,1.30,4000.0,0,NaN,1,16,6.50,60,3,android,8.0,8.0,1,720,1600
727,tecno,Tecno Spark 9 Pro,9999,7.4,0,helio,8.0,2.00,5000.0,1,18.0,4,64,6.60,90,3,android,48.0,32.0,1,2460,1080
120,infinix,Infinix Note 11S (8GB RAM + 128GB),15999,7.9,0,helio,8.0,2.05,5000.0,1,33.0,8,128,6.95,120,3,android,50.0,16.0,1,2460,1080
104,huawei,Huawei P40 Pro,82990,NaN,1,kirin,8.0,2.86,4200.0,1,40.0,8,256,6.58,90,4,android,50.0,32.0,1,2640,1200
605,samsung,Samsung Galaxy A74 5G,42999,NaN,1,snapdragon,8.0,2.36,5000.0,1,33.0,8,128,6.70,120,4,android,108.0,32.0,1,2412,1080


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 980 entries, 0 to 979
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   brand_name                 980 non-null    object 
 1   model                      980 non-null    object 
 2   price                      980 non-null    int64  
 3   avg_rating                 879 non-null    float64
 4   5G_or_not                  980 non-null    int64  
 5   processor_brand            960 non-null    object 
 6   num_cores                  974 non-null    float64
 7   processor_speed            938 non-null    float64
 8   battery_capacity           969 non-null    float64
 9   fast_charging_available    980 non-null    int64  
 10  fast_charging              769 non-null    float64
 11  ram_capacity               980 non-null    int64  
 12  internal_memory            980 non-null    int64  
 13  screen_size                980 non-null    float64

###### brand_name,model,processor_brand,os are the objects and rest are numeric .There are some null or missing values present in the dataset.

### EXPLORATORY DATA ANALYSIS

In [18]:
df.describe()

,price,avg_rating,5G_or_not,num_cores,processor_speed,battery_capacity,fast_charging_available,fast_charging,ram_capacity,internal_memory,screen_size,refresh_rate,num_rear_cameras,primary_camera_rear,primary_camera_front,extended_memory_available,resolution_height,resolution_width
count,980.000000,879.000000,980.000000,974.000000,938.000000,969.000000,980.000000,769.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,975.000000,980.000000,980.000000,980.000000
mean,32520.504082,7.825825,0.560204,7.772074,2.427217,4817.748194,0.854082,46.126138,6.560204,141.036735,6.536765,92.256122,2.814286,50.319286,16.589744,0.630612,2214.663265,1075.852041
std,39531.812669,0.740285,0.496616,0.836845,0.464090,1009.540054,0.353205,34.277870,2.744378,107.134516,0.349162,28.988052,0.776441,33.000968,10.876944,0.482885,516.484254,290.164931
min,3499.000000,6.000000,0.000000,4.000000,1.200000,1821.000000,0.000000,10.000000,1.000000,8.000000,3.540000,60.000000,1.000000,2.000000,0.000000,0.000000,480.000000,480.000000
25%,12999.000000,7.400000,0.000000,8.000000,2.050000,4500.000000,1.000000,18.000000,4.000000,64.000000,6.500000,60.000000,2.000000,24.000000,8.000000,0.000000,1612.000000,1080.000000
50%,19994.500000,8.000000,1.000000,8.000000,2.300000,5000.000000,1.000000,33.000000,6.000000,128.000000,6.580000,90.000000,3.000000,50.000000,16.000000,1.000000,2400.000000,1080.000000
75%,35491.500000,8.400000,1.000000,8.000000,2.840000,5000.000000,1.000000,66.000000,8.000000,128.000000,6.670000,120.000000,3.000000,64.000000,16.000000,1.000000,2408.000000,1080.000000
max,650000.000000,8.900000,1.000000,8.000000,3.220000,22000.000000,1.000000,240.000000,18.000000,1024.000000,8.030000,240.000000,4.000000,200.000000,60.000000,1.000000,3840.000000,2460.000000


##### Not all the smartphones are newly invented model as some of the phones have 1 rear camera and some in fact do not have front cameras. Price are from 3500-650000(approx,there is a large variation in our collected data). There are a lot of binary responses like 5G or not,fast charging availability extended memory capacity(0 for no and 1 for yes). Primary rear camera varies from 2 MP -200 MP.

In [20]:
df.describe(include="object")

,brand_name,model,processor_brand,os
count,980,980,960,966
unique,46,980,13,3
top,xiaomi,Apple iPhone 11,snapdragon,android
freq,134,1,413,909


###### We have 46 unique brands with 980 models,3 different operating system.  Majority of the observations are android phones followed by snapdragon,xiaomi,Apple iPhone 11.

In [21]:
df.describe(include="int64")

,price,5G_or_not,fast_charging_available,ram_capacity,internal_memory,refresh_rate,num_rear_cameras,extended_memory_available,resolution_height,resolution_width
count,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000
mean,32520.504082,0.560204,0.854082,6.560204,141.036735,92.256122,2.814286,0.630612,2214.663265,1075.852041
std,39531.812669,0.496616,0.353205,2.744378,107.134516,28.988052,0.776441,0.482885,516.484254,290.164931
min,3499.000000,0.000000,0.000000,1.000000,8.000000,60.000000,1.000000,0.000000,480.000000,480.000000
25%,12999.000000,0.000000,1.000000,4.000000,64.000000,60.000000,2.000000,0.000000,1612.000000,1080.000000
50%,19994.500000,1.000000,1.000000,6.000000,128.000000,90.000000,3.000000,1.000000,2400.000000,1080.000000
75%,35491.500000,1.000000,1.000000,8.000000,128.000000,120.000000,3.000000,1.000000,2408.000000,1080.000000
max,650000.000000,1.000000,1.000000,18.000000,1024.000000,240.000000,4.000000,1.000000,3840.000000,2460.000000


In [22]:
df.describe(include="float")

,avg_rating,num_cores,processor_speed,battery_capacity,fast_charging,screen_size,primary_camera_rear,primary_camera_front
count,879.000000,974.000000,938.000000,969.000000,769.000000,980.000000,980.000000,975.000000
mean,7.825825,7.772074,2.427217,4817.748194,46.126138,6.536765,50.319286,16.589744
std,0.740285,0.836845,0.464090,1009.540054,34.277870,0.349162,33.000968,10.876944
min,6.000000,4.000000,1.200000,1821.000000,10.000000,3.540000,2.000000,0.000000
25%,7.400000,8.000000,2.050000,4500.000000,18.000000,6.500000,24.000000,8.000000
50%,8.000000,8.000000,2.300000,5000.000000,33.000000,6.580000,50.000000,16.000000
75%,8.400000,8.000000,2.840000,5000.000000,66.000000,6.670000,64.000000,16.000000
max,8.900000,8.000000,3.220000,22000.000000,240.000000,8.030000,200.000000,60.000000


###### As 5G network is not available tilll now in most of the places of our world so majority of the phones are not 5G.